# Analysons les ventes de ton entreprise 📚📚

In [7]:
import re
import pandas as pd
from datetime import datetime 
import numpy as np

 ## 1. Cleaning files

#### Fichier "Customers"

In [8]:
#open and read csv file
customers_df = pd.read_csv("../datas/dataset_bookstore_stats/customers.csv")
display(customers_df)

,client_id,sex,birth
0,c_4410,f,1967
1,c_7839,f,1975
2,c_1699,f,1984
3,c_5961,f,1962
4,c_5320,m,1943
...,...,...,...
8618,c_7920,m,1956
8619,c_7403,f,1970
8620,c_5119,m,1974
8621,c_5643,f,1968


In [9]:
doublons = customers_df['client_id'].duplicated()

# Comptez le nombre de doublons
nombre_de_doublons = doublons.sum()

# Affichez le résultat
if nombre_de_doublons == 0:
    print("Il n'y a pas de valeurs identiques dans la colonne.")
else:
    print(f"Il y a {nombre_de_doublons} valeur(s) identique(s) dans la colonne.")
    print("Les doublons sont aux index suivants:")
    print(customers_df[doublons].index.tolist())

Il n'y a pas de valeurs identiques dans la colonne.


In [10]:
#delete possible space into my column
customers_df['client_id'] = customers_df['client_id'].str.replace(' ', '')
customers_df['sex'] = customers_df['sex'].str.replace(' ', '')
customers_df['birth'] = customers_df['birth'].astype(str).str.replace(' ', '').astype(int)

In [11]:
#To check max and min value in my column birth
display(customers_df['birth'].max())
display(customers_df['birth'].min())

2004

1929

In [12]:
# Utilisez la méthode unique() pour obtenir les valeurs uniques dans la colonne 'genre'
valeurs_uniques = customers_df['sex'].unique()

# Vérifiez si toutes les valeurs uniques sont soit 'f' soit 'm'
valeurs_autorisees = {'f', 'm'}
if set(valeurs_uniques) <= valeurs_autorisees:
    print("La colonne ne comprend que les valeurs 'f' ou 'm'.")
else:
    print("La colonne contient des valeurs autres que 'f' ou 'm'.")

La colonne ne comprend que les valeurs 'f' ou 'm'.


#### Fichier "Products"

In [13]:
#open and read csv file
products_df = pd.read_csv("../datas/dataset_bookstore_stats/products.csv")
display(products_df)

,id_prod,price,categ
0,0_1421,19.99,0
1,0_1368,5.13,0
2,0_731,17.99,0
3,1_587,4.99,1
4,0_1507,3.99,0
...,...,...,...
3282,2_23,115.99,2
3283,0_146,17.14,0
3284,0_802,11.22,0
3285,1_140,38.56,1


In [14]:
#check type of each column
print('Colonne id_prod :', products_df['id_prod'].dtype)
print('Colonne prix :', products_df['price'].dtype)
print('Colonne categ :', products_df['categ'].dtype)

Colonne id_prod : object
Colonne prix : float64
Colonne categ : int64


In [15]:
# delete possible space
products_df['id_prod'] = products_df['id_prod'].str.replace(' ', '')
products_df['price'] = products_df['price'].astype(str).str.replace(' ', '').astype(float)
products_df['categ'] = products_df['categ'].astype(str).str.replace(' ', '').astype(int)

In [16]:
display(products_df['price'].max())
display(products_df['price'].min())

300.0

-1.0

In [17]:
lignes_floats_negatifs = products_df[products_df['price'] < 0]
display(lignes_floats_negatifs)

,id_prod,price,categ
731,T_0,-1.0,0


In [18]:
#Delete the result I've got before, so no negative price into my column
index_a_supprimer = lignes_floats_negatifs.index
products_df.drop(index_a_supprimer, inplace=True)

In [19]:
display(products_df['categ'].max())
display(products_df['categ'].min())

2

0

In [20]:
# Filtrer les lignes où id_prod[0] est différent de categ
mismatched_rows = products_df.loc[products_df['id_prod'].astype(str).str[0] != products_df['categ'].astype(str)]

# Afficher les lignes où la vérification a échoué
display(mismatched_rows.count())

id_prod    0
price      0
categ      0
dtype: int64

#### Fichier "Transactions" (notre table d'association)

In [21]:
deals_df = pd.read_csv("../datas/dataset_bookstore_stats/transactions.csv")
display(deals_df)

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242
...,...,...,...,...
337011,1_671,2021-05-28 12:35:46.214839,s_40720,c_3454
337012,0_759,2021-06-19 00:19:23.917703,s_50568,c_6268
337013,0_1256,2021-03-16 17:31:59.442007,s_7219,c_4137
337014,2_227,2021-10-30 16:50:15.997750,s_112349,c_5


In [22]:
doublons = deals_df['session_id'].duplicated()

# Comptez le nombre de doublons
nombre_de_doublons = doublons.sum()

# Affichez le résultat
if nombre_de_doublons == 0:
    print("Il n'y a pas de valeurs identiques dans la colonne.")
else:
    print(f"Il y a {nombre_de_doublons} valeur(s) identique(s) dans la colonne.")
    print("Les doublons sont aux index suivants:")
    print(deals_df[doublons].index.tolist())

Il y a 167821 valeur(s) identique(s) dans la colonne.
Les doublons sont aux index suivants:
[563, 852, 919, 1422, 1427, 1613, 1764, 1780, 1955, 2106, 2147, 2365, 2376, 2414, 2472, 2736, 2754, 2756, 2768, 2895, 2964, 3004, 3006, 3184, 3387, 3428, 3484, 3569, 3577, 3647, 3672, 3677, 3715, 3727, 3736, 3738, 3747, 3765, 3874, 3921, 3923, 3927, 3957, 3995, 4000, 4015, 4035, 4041, 4099, 4128, 4169, 4184, 4205, 4207, 4376, 4406, 4470, 4493, 4568, 4602, 4603, 4677, 4692, 4718, 4753, 4770, 4786, 4834, 4866, 4872, 5015, 5030, 5047, 5122, 5149, 5172, 5243, 5247, 5248, 5252, 5258, 5271, 5297, 5361, 5509, 5529, 5544, 5614, 5632, 5746, 5830, 5854, 5955, 5972, 5978, 5991, 5998, 6057, 6110, 6246, 6251, 6270, 6289, 6306, 6356, 6361, 6395, 6438, 6467, 6486, 6509, 6523, 6541, 6596, 6644, 6650, 6684, 6740, 6776, 6792, 6794, 6799, 6807, 6830, 6888, 6899, 6973, 7014, 7049, 7086, 7126, 7131, 7133, 7144, 7180, 7191, 7247, 7277, 7283, 7294, 7324, 7338, 7374, 7411, 7415, 7439, 7445, 7455, 7496, 7537, 7560, 7585

In [23]:
group_by_sessions = deals_df['session_id'].value_counts()
display(group_by_sessions)

s_0         200
s_118668     14
s_96857      13
s_21005      13
s_168560     12
           ... 
s_26281       1
s_73468       1
s_169637      1
s_131653      1
s_112349      1
Name: session_id, Length: 169195, dtype: int64

In [24]:
filtered_df = deals_df[deals_df['session_id'] == 's_0']

# Afficher le résultat
print(filtered_df)

       id_prod                             date session_id client_id
1431       T_0  test_2021-03-01 02:30:02.237420        s_0      ct_1
2365       T_0  test_2021-03-01 02:30:02.237446        s_0      ct_1
2895       T_0  test_2021-03-01 02:30:02.237414        s_0      ct_1
5955       T_0  test_2021-03-01 02:30:02.237441        s_0      ct_0
7283       T_0  test_2021-03-01 02:30:02.237434        s_0      ct_1
...        ...                              ...        ...       ...
332594     T_0  test_2021-03-01 02:30:02.237445        s_0      ct_0
332705     T_0  test_2021-03-01 02:30:02.237423        s_0      ct_1
332730     T_0  test_2021-03-01 02:30:02.237421        s_0      ct_1
333442     T_0  test_2021-03-01 02:30:02.237431        s_0      ct_1
335279     T_0  test_2021-03-01 02:30:02.237430        s_0      ct_0

[200 rows x 4 columns]


In [25]:
#I can see that all session 's_0' is associated to an id_product 'T_O'
# Delete all rows with 'T_0'

deals_df = deals_df.drop(filtered_df.index)

# Si vous voulez réinitialiser l'index du DataFrame après la suppression
deals_df.reset_index(drop=True, inplace=True)

display(deals_df)

,id_prod,date,session_id,client_id
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242
...,...,...,...,...
336811,1_671,2021-05-28 12:35:46.214839,s_40720,c_3454
336812,0_759,2021-06-19 00:19:23.917703,s_50568,c_6268
336813,0_1256,2021-03-16 17:31:59.442007,s_7219,c_4137
336814,2_227,2021-10-30 16:50:15.997750,s_112349,c_5


In [26]:
# delete possible space
deals_df['id_prod'] = deals_df['id_prod'].str.replace(' ', '')
deals_df['session_id'] = deals_df['session_id'].str.replace(' ', '')
deals_df['client_id'] = deals_df['client_id'].str.replace(' ', '')

In [27]:
# Vérifier que toutes les datas dans la colonne 'session_id' commencent par 's_'
all_start_with_s = deals_df['session_id'].str.startswith('s_').all()

if all_start_with_s:
    print("Toutes les datas dans la colonne 'session_id' commencent par 's_'.")
else:
    print("Au moins une data dans la colonne 'session_id' ne commence pas par 's_'.")

Toutes les datas dans la colonne 'session_id' commencent par 's_'.


In [28]:
# Vérifier que toutes les datas dans la colonne 'client_id' commencent par 'c_'
all_start_with_c = deals_df['client_id'].str.startswith('c_').all()

if all_start_with_c:
    print("Toutes les datas dans la colonne 'client_id' commencent par 'c_'.")
else:
    print("Au moins une data dans la colonne 'client_id' ne commence pas par 'c_'.")

Toutes les datas dans la colonne 'client_id' commencent par 'c_'.


 ## 2. Seconde étape

### a) Indicateurs de tendance centrale et de dispersion

#### Exemples d'indicateurs de tendance centrale

In [29]:
#save modifications to csv
customers_df.to_csv('../datas/dataset_bookstore_stats/customers.csv', index=False)
deals_df.to_csv('../datas/dataset_bookstore_stats/transactions.csv', index=False)
products_df.to_csv('../datas/dataset_bookstore_stats/products.csv', index=False)